Before we start running our scripts I would like to do a small discussion about this question.

Personally, I had a problem to understand the scope of this question that arises from 2 major things.

1. I could not understand how we can use anova to compare proportions. Especially, after some research on the internet, none was suggesting anova for proportion comparison. Also, with anova, we can't do one-tail tests(i.e. Finding if a group is statistically significant bigger than the other one).

2. My second missunderstanding was the comparison between which groups we should select. It was not clear for my understanding if I had to compare male-seniors with male-non_seniors or male-seniors with female-seniors. Similarly if I had to compare white-seniors with white-non_seniors or white-seniors with other_race-seniors.

Because of the uncertainty I had, I chose to implement all the above cases.

To conclude, I believe that the best interpretation and implementation is:

Question1: proportion_ztest with:

H0: proportion_male_seniors = proportion_male_non_seniors

H1: proportion_male_seniors > proportion_male_non_seniors
                                 
Question2: proportion_ztest with:

H0: proportion_white_seniors = proportion_white_non_seniors

H1: proportion_white_seniors > proportion_white_non_seniors

In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as  plt
import statsmodels as st
import statsmodels.formula.api as smf
import statsmodels.api as sm
%matplotlib inline

data_filename = 'C:/Users/User/Desktop/Practical_DS_Leon/Assign_3/data/Reveal_EEO1_for_2016.csv'
df = pd.read_csv(data_filename)

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Question 1, Case 1:
We will compare MALE-SENIORS with MALE-NON_SENIORS

In [2]:
#(A)
#ETL proccess stage_1
#-select only the columns we need and
#-drop NaNs 
#-exclude Totals and Previous Totals
dfA = df.loc[:,['gender', 'job_category','count']].dropna().copy()
dfA = dfA[dfA.job_category != 'Totals']
dfA = dfA[dfA.job_category != 'Previous_totals']

In [3]:
#ETL proccess stage_2
#-select only the males because the case is to find if the proportion of males in senior positions is greater than that in non-senior positions!
#-exclude count equal zero
dfA = dfA[dfA['gender'] == 'male']
dfA = dfA[dfA['count'] !='0']
#-case count as integer
dfA['count'] = dfA['count'].astype(int)

In [4]:
#ETL proccess stage_3
#finally transform the DF to the final form that we will run ANOVA (similar with the example on the lectures)
dfA['job_category'] = dfA.apply(lambda row: 'senior' if row.job_category =='Executive/Senior officials & Mgrs' \
                        else 'non_senior', axis = 1)

In [5]:
#p-value is 0.023. For a = 0.05 we reject H0 that says the two categories (senior-non_senior) employ the same ammount of people
lm = smf.ols('count ~ C(job_category)', data=dfA).fit()
table = sm.stats.anova_lm(lm, test = 'F', typ = 2 )
table

,sum_sq,df,F,PR(>F)
C(job_category),9.693892e+06,1.0,5.154919,0.023471
Residual,1.374655e+09,731.0,NaN,NaN


GENERAL DISCUSSION ANOVA:


ANOVA is used to compare differences of means among more than 2 groups. It does this by looking at variation in the data and where that variation is found (hence its name). Specifically, ANOVA compares the amount of variation between groups with the amount of variation within groups.

First of all, we should mension that anova for only two groups is equal t.test and we should take the same result
Even though, with t.test we can run one-tail test to examine if the proportion of seniors is statistically greater than the non-seniors and not only if the two proportions are  statistically significant different in general.

MORE EXPLANATION FOR THE ANOVA TABLE
df = degrees of freedom
df between groups = k-1 = 2 - 1 = 1  (the number of groups minus 1)
df between within groups = n-k = 733 - 2 = 731 (the number of observations minus the number of groups)

sum_sq = sum of squares
1rst row: The between-group variation (or between-group sums of squares, BSS) is calculated by comparing the mean of each group with the overall mean of the data. 
2nd row: The within-group variation (or the within-group sums of squares, ) is the variation of each observation from its group mean.

mean_sq
1rst row: Mean variation between groups. (Divide the Between-SS by the number of degrees of freedom)
2nd row : Mean variation within groups. (Divede Within-SS by the total degrees of freedom)

F-ratio = mean_variation_betwen_groups / mean_variation_within_groups
If the average difference between groups is similar to that within groups, the F ratio is about 1. As the average difference between groups becomes greater than that within groups, the F ratio becomes larger than 1.

P-value: 
We use F-distribution's table to find the corresponding pvalue.
Larger F-ratios gives smaller P-values.
For significant level of 0.05 we want a p-value less than 0.05 to reject the H_0 (says that the two categories (senior-non_senior) employ the same ammount of people).

ANOTHER APROACH TO THE PROBLEM (personally I think it is better but maybe I am totally wrong)

We can test if the percentage of seniors is statistically significant different than non-seniors
Let's try this out...

In [6]:
#Transform the data
df_counts = dfA.groupby('job_category').sum()
#count: the number of successes in nobs trials
count = np.array([df_counts.iloc[1], df_counts.iloc[0]]) 
#nobs: the number of trials or observations, with the same length as sum(count).
nobs = np.array([df_counts['count'].sum()])

In [7]:
#DOCUMENTATION : http://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportions_ztest.html
#H0: p_seniors = p_non_seniors
#H1: p_seniors > p_non_seniors
z_score, p_value = sm.stats.proportions_ztest(count, nobs,alternative='larger')
print(p_value)
#obviously we not reject the H0 that says that the proportion of senior males same than non-seniors compared to h1 says that
#seniors is greater than non_seniors

[ 1.]


Question 1: Case 2: We will compare MALE-SENIORS with FEMALE-SENIORS

In [8]:
#ETL proccess
dfB = df.loc[:,['gender', 'job_category','count']].dropna().copy()
dfB = dfB[dfB.job_category != 'Totals']
dfB = dfB[dfB.job_category != 'Previous_totals']

dfB['count'] = dfB['count'].astype(int)

dfB['job_category'] = dfB.apply(lambda row: 'senior' if row.job_category =='Executive/Senior officials & Mgrs' \
                        else 'non_senior', axis = 1)

#prepare data for zproportion_test
female_seniors = dfB.loc[(dfB['gender']=='female') & (dfB['job_category']=='senior'),['count']].sum()
male_seniors = dfB.loc[(dfB['gender']=='male') & (dfB['job_category']=='senior'),['count']].sum()
counts = np.array(male_seniors, female_seniors)
female_non_seniors = dfB.loc[(dfB['gender']=='female') & (dfB['job_category']=='non_senior'),['count']].sum()
male_non_seniors = dfB.loc[(dfB['gender']=='male') & (dfB['job_category']=='non_senior'),['count']].sum()
nobs =  np.array(female_non_seniors,male_non_seniors)

dfB = dfB[dfB['job_category'] == 'senior']
dfB = dfB[dfB['count'] != 0]

In [9]:
lm_gender = smf.ols('count ~ C(gender)', data=dfB).fit()
table = sm.stats.anova_lm(lm_gender, test = 'F', typ = 2 )
table
#Statistically significant difference between male_seniors-female_seniors

,sum_sq,df,F,PR(>F)
C(gender),12795.256566,1.0,4.824174,0.029558
Residual,408457.410101,154.0,NaN,NaN


In [10]:
z_score, p_value = sm.stats.proportions_ztest([male_seniors, female_seniors], [male_non_seniors, female_non_seniors], alternative='larger')
print(p_value)
#reject h0 that says that the proportion of male_seniors = female_seniors compared to h1 that says that the proportion of male_seniors 
#is greater than female_seniors

[  3.37912441e-24]


Question 2: Case 1: We will compare WHITE-SENIORS with WHITE-NON-SENIOR

In [33]:
#(B)
#ETL
#The second question is similar with the first one.
#Instead of selecting all the males this time we will select all whites
#We will follow the same procedure. No further explanation needs here, its exactly the same case as above.
dfC = df.loc[:,['race', 'job_category','count']].dropna().copy()
dfC = dfC[dfC.job_category != 'Totals']
dfC = dfC[dfC.job_category != 'Previous_totals']
dfC = dfC[dfC['race'] == 'White']
dfC = dfC[dfC['count'] !='0']
dfC['count'] = dfC['count'].astype(int)

dfC['job_category'] = dfC.apply(lambda row: 'senior' if row.job_category =='Executive/Senior officials & Mgrs' \
                        else 'non_senior', axis = 1)

white_seniors = dfC.loc[(dfC['race']=='White') & (dfC['job_category']=='senior'), 'count'].sum()
white_non_seniors = dfC.loc[(dfC['race']=='White') & (dfC['job_category']=='non_senior'), 'count'].sum()
all_whites = white_seniors + white_non_seniors

In [19]:
#ANOVA
#p-value is 0.01. One more time, for a = 0.05 we reject H0 that says the two categories (senior/non_senior)
#employ the same ammount of WHITE people
lm = smf.ols('count ~ C(job_category)', data=dfC).fit()
table = sm.stats.anova_lm(lm, test = 'F', typ = 2 )
table

,sum_sq,df,F,PR(>F)
C(job_category),2.639122e+07,1.0,6.655966,0.010424
Residual,1.046772e+09,264.0,NaN,NaN


Question 2: Case 2: We will compare WHITE-SENIORS with ALL-OTHER-SENIORS

In [38]:
#ETL proccess
dfD = df.loc[:,['race', 'job_category','count']].dropna().copy()
dfD = dfD[dfD.job_category != 'Totals']
dfD = dfD[dfD.job_category != 'Previous_totals']

dfD['count'] = dfD['count'].astype(int)

dfD['job_category'] = dfD.apply(lambda row: 'senior' if row.job_category =='Executive/Senior officials & Mgrs' \
                        else 'non_senior', axis = 1)

dfD['race'] = dfD.apply(lambda row: 'White' if row.race =='White' \
                        else 'other', axis = 1)

#prepare data for zproportion_test
white_seniors = dfD.loc[(dfD['race']=='White') & (dfD['job_category']=='senior'),['count']].sum()
other_seniors = dfD.loc[(dfD['race']=='other') & (dfD['job_category']=='senior'),['count']].sum()
counts = np.array(white_seniors, other_seniors)
white_non_seniors = dfD.loc[(dfD['race']=='White') & (dfD['job_category']=='non_senior'),['count']].sum()
other_non_seniors = dfD.loc[(dfD['race']=='other') & (dfD['job_category']=='non_senior'),['count']].sum()
nobs =  np.array(white_non_seniors, other_non_seniors)

dfD = dfD[dfD['job_category'] == 'senior']
dfD = dfD[dfD['count'] != 0]

In [40]:
#ANOVA
#p-value is 0.075. For a = 0.05 we do not reject H0 that says the two categories (white/other)
#employ the same ammount of senior people
lm = smf.ols('count ~ C(race)', data=dfD).fit()
table = sm.stats.anova_lm(lm, test = 'F', typ = 2 )
table

,sum_sq,df,F,PR(>F)
C(race),2.548133e+04,1.0,3.200931,0.075316
Residual,1.401066e+06,176.0,NaN,NaN


In [42]:
z_score, p_value = sm.stats.proportions_ztest([white_seniors, other_seniors], [white_non_seniors, other_non_seniors], alternative='larger')
print(p_value)
#we reject the H0 that the 2 groups have the same proportion of people as seniors and non-seniors with H1: proportion_white_senior > proportion_other_senior


[  4.79590353e-50]
